In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import tensorflow as tf
#import keras.preprocessing.image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout , BatchNormalization
from keras.optimizers import Adam, Adamax
from keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

training = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/train'
validation = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid'
test = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/test'


train_dataset = tf.keras.utils.image_dataset_from_directory(
    training,
    labels='inferred',             # Automatically infers labels from folder names
    label_mode='categorical',              # Options: 'int', 'categorical', 'binary', or None
    batch_size=32,                 # Number of images per batch
    image_size=(256, 256),         # Resize images to this size
    shuffle=True,                  # Shuffle the dataset
    seed=123                       # Random seed for reproducibility
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    validation,
    labels='inferred',             
    label_mode='categorical',             
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    seed=123)                     

print(train_dataset.class_names)


In [ ]:
num_classes = 4

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'softmax')
    ]
)

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [ ]:
cnn_model = model.fit(
      train_dataset,
      validation_data=val_dataset,
      epochs=30
    )

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test,
    labels='inferred',             
    label_mode='categorical',             
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    seed=123)                     

In [ ]:
y_true = []
y_pred = []

for images, labels in test_dataset:
    predictions = model.predict(images, verbose=0)

    # Handle one-hot encoded labels or integer labels
    if len(labels.shape) > 1 and labels.shape[1] > 1:
        labels = tf.argmax(labels, axis=1)  # Convert one-hot to class index

    preds = tf.argmax(predictions, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds.numpy())

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
print(cm)

from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred,target_names = ['overripe','ripe','rotten','underripe']))

In [ ]:
import pickle

with open('/kaggle/working/bananas_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
with open('/kaggle/working/bananas_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Use the model
predictions = loaded_model.predict(test_dataset)
print(predictions[:1])
label = np.argmax(predictions[:1], axis=1)[0]
print(label)